# crawler tutorial

这篇文章将主要讲述 python 爬虫的基本用法以及实操建议，主要内容包括：

• 最基本的 HTTP 知识以及如何利用浏览器 inspect 功能

• 解爬取网页的两大方式：利用 requests 发送请求，以及利用 selenium 模拟浏览器访问网页

• 实际操作过程中的注意事项

• 以不同类型的网页为例讲解什么情况下具体应当使用哪种方式

# 一、前置知识
在爬取网页之前，首先需要简单了解 HTTP 基本知识，以及浏览器 inspect 功能的用法。

## HTTP 基本知识



### 1.什么是 HTTP

HTTP:Hypertext Transfer Protocol（超文本传输协议）是客户端与服务端之间的一个简单的请求-响应协议。

Example:

客户端（浏览器）向服务端发送了一个 HTTP 请求(比如 get)，服务端则返回客户端一个响应(response)。响应中包含了响应的状态信息，并且可能携带了所请求的内容。



### 2.HTTP 方法

最常用的 HTTP 方法为 get 和 post。

爬虫中需要用到的是 get 方法，get 方法即是向特定的信息源请求数据。

而 post 则是向一个特定的服务端发送数据。



### 3.HTTP 响应状态码

HTTP 响应状态码被用于表明特定的 HTTP 请求是否完成/错误原因。响应有以下 5 类：

• 信息响应 (100–199)

• 成功响应 (200–299)

• 重定向消息 (300–399)

• 客户端错误响应 (400–499)

• 服务端错误响应 (500–599)

值得注意的响应码有：

200: 响应成功，爬虫中，get 请求的 status code 为 200，即请求成功。但注意请求成功不代表正确返回所需内容（返回的也有可能是百度安全验证）。

404: not found。该状态码表明服务器上无法找到请求的资源。除此之外，也可以在服务器端拒绝请求且不想说明理由时使用。

429: too many requests 用户在给定的时间内发送了太多请求（"限制请求速率"）。这也是常见的反爬手段之一，如果频繁向 google search 请求信息，谷歌很快会开始返回 429。所以，爬取的过程中注意 sleep，将程序停止一段时间。

更详细的 status code 信息请参见：https://developer.mozilla.org/zh-CN/docs/Web/HTTP/Status。



### 4.Cookie

HTTP Cookie（也叫 Web Cookie 或浏览器 Cookie）是服务器发送到用户浏览器并保存在本地的一小块数据，它会在浏览器下次向同一服务器再发起请求时被携带并发送到服务器上。通常，它用于告知服务端两个请求是否来自同一浏览器，如保持用户的登录状态。

Cookie 主要用于以下三个方面：

- 会话状态管理（如用户登录状态、购物车、游戏分数或其它需要记录的信息）
- 个性化设置（如用户自定义设置、主题等）
- 浏览器行为跟踪（如跟踪分析用户行为等）

爬取网页时，有些网页需要登录才能看到具体内容。这种情况下get请求需要传入cookie来表明你登录了。



### 5.User-Agent

User-Agent 即用户代理，简称“UA”，它是一个特殊字符串头。网站服务器通过识别“UA”来确定用户所使用的操作系统版本、CPU 类型、浏览器版本等信息。而网站服务器则通过判断 UA 来给客户端发送不同的页面。

比如 Mac 上 Chrome 的 User-Agent 就是：`Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36`。

爬虫使用代码来访问网站，其在进行 get 请求时可能不带有 user-agent 信息。网站可以通过识别请求头中 User-Agent 信息来判断是否是爬虫访问网站。如果是，网站首先对该 IP 进行预警，对其进行重点监控，当发现该 IP 超过规定时间内的访问次数， 将在一段时间内禁止其再次访问网站。





# 二、爬取网页的两大方式
爬虫的两大流派为 requests 与 selenium。其中 requests 为直接在程序中请求数据，而 selenium 则会模拟浏览器访问页面。
本部分将以爬取不同网站作为例子来讲解两种方式。

## requests

本部分将主要讲解 requests.get()方法，以及为了请求到所想要的数据所需传入的参数，包括 headers 中的 user-agent, cookie，以及 proxies 参数。

使用 requests 需要先 import requests 库。

In [1]:
import requests

### 1. requests.get

最基本地，利用 `requests.get(URL)` 即可以请求所需要的信息。`requests.get()` 即是通过以上讲的HTTP get方法来向服务端请求信息。

比如，我们试着请求天涯杂谈的首页信息:

In [2]:
resp = requests.get("http://bbs.tianya.cn/list-free-1.shtml")

我们可以查看一下这个 requests 所返回的网页源码：

In [3]:
resp.text

'\r\n<!DOCTYPE HTML>\r\n<html class="bbs-page">\r\n<head>\r\n<meta charset="utf-8">\r\n<title>天涯杂谈_论坛_天涯社区</title>\r\n<meta name="keywords" content="天涯杂谈,社会,天涯,天涯论坛,天涯社区" />\r\n<meta name="description" content="天涯社区旗下天涯杂谈论坛版块，共有5250030个主帖，152200757个回帖，1578205个成员。天涯杂谈是天涯社区人气、流量、影响力最大的板块之一，集自由、多元、包容为一体。倡导关注自我、关注人性、关注社会民生，反应灵敏、传递迅速、观点明晰，对现实有着舆论监督和振聋发聩的作用" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n\r\n<meta http-equiv="Cache-Control" content="no-transform"/>\r\n<meta http-equiv="Cache-Control" content="no-siteapp"/> \r\n<meta name="applicable-device" content="pc">\r\n<meta name="mobile-agent" content="format=xhtml; url=http://bbs.tianya.cn/m/list-free-1.shtml">\r\n<meta name="mobile-agent" content="format=html5; url=http://bbs.tianya.cn/m/list-free-1.shtml">\r\n<meta name="mobile-agent" content="format=wml; url=http://bbs.tianya.cn/m/list-free-1.shtml">\r\n<link rel="alternate" media="only screen and (max-width: 640px)" href="http://bbs.tianya.cn/m/list-free-1.shtml"

恭喜你！这样你就成功完成了一个请求！如果你的任务是从该页面中提取信息，那么爬取的任务就这么简单地完成了 :P。接下来就是解析这个网页了。



### 2. user-agent

但是，有的网页直接 get 信息会失败，比如访问豆瓣：

In [4]:
resp = requests.get("https://www.douban.com/")

In [5]:
resp

<Response [418]>

没有能正确返回信息。

我们再来看看 resp 的 headers,可以发现没有 user-agent。

In [6]:
resp.request.headers

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

User-Agent 为 python-requests/2.27.1，可以看到这是明摆着自己是爬虫，不拒绝你拒绝谁(x

我们在上面讲过，网页可能通过 user-agent 和 cookie 来判断来访者是不是网络爬虫，这里就是因为 get 请求没有带上正常的 user-agent 导致请求被拒绝，所以：

In [7]:
resp = requests.get("https://www.douban.com/",
                    headers={"user-agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})

In [8]:
resp

<Response [200]>

加上 user-agent，豆瓣就能返回信息了。

user-agent 可以通过上述的 inspect 方法获取，但是有一种更好的方式，不仅能免去复制长传字符的烦恼，还能通过随机选择 user-agent 来防止网页拒绝来自同一个 user-agent 的大量请求。这就是 fake_useragent 库的作用。比如不断运行下面的代码，可以得到不同的 user-agent。

In [40]:
from fake_useragent import UserAgent as ua
print(ua(use_cache_server=False).random)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


所以爬取的时候可以无脑写下 `headers={ua(use_cache_server=False).random}` 这样的代码，简单好用，何乐而不为。

注：use_cache_server=False 为禁用浏览器缓存。


### 3.cookie

上面我们讲过，网站可以通过 cookies 来判断用户是否登录，登录了才会返回更多的信息。所以，我们可以通过传入 cookie 来获取登录后的信息。cookie 的获取方式同上所述。

比如 bilibili 只有登录了才会显示阿婆主的 likes 以及 view。我们通过 inspect 功能找到请求 likes 数据的包，得到了"https://api.bilibili.com/x/space/upstat?mid=13638568&jsonp=jsonp"。我们通过 requests.get() 进行请求。

In [21]:
resp = requests.get("https://api.bilibili.com/x/space/upstat?mid=13638568&jsonp=jsonp" , headers={
    "User-Agent": ua(use_cache_server=False).random})

In [22]:
resp.text

'{"code":0,"message":"0","ttl":1,"data":{}}'

但是如果带上了 cookie（小练习-请在这里查找并填入你自己的cookie）:

In [23]:
cookie = ""
headers = {"user-agent": ua(use_cache_server=False).random, "cookie": cookie}
resp = requests.get("https://api.bilibili.com/x/space/upstat?mid=13638568&jsonp=jsonp" , headers=headers)

In [24]:
resp.text

'{"code":0,"message":"0","ttl":1,"data":{}}'

就能正确显示信息啦！



### 4.proxies

上面提到，如果反复对同一个网站发起请求，你的 ip 会被网页 ban 掉，比如 google 会返回 429，百度返回百度安全验证网页。这种情况有的时候可以通过增大 time.sleep() 的时间，即程序中止的时间来解决，但有的时候即便 sleep 值很大仍然会发生各种各样的错误，所以可能需要 ip 池。使用 ip 池则可以不用 sleep。

proxies 的参数可以被这样传入：

In [ ]:
cookie = ""
ip = ""
port = ""
proxies={'http': f'{ip}:{port}'}
resp = requests.get("http://bbs.tianya.cn/list-free-1.shtml" , headers=headers, proxies=proxies)

具体教程请自行善用搜索。


## selenium

有的时候，网页会动态返回内容。比如，你翻页翻到下方才能看到全部的视频评论/帖子。所以这个时候直接用 requests.get() 无法获得所有内容，可以通过 selenium 自动登录/翻页来获取所有内容。

与利用 requests 时不同，利用 selenium 你需要做的事情就是思考如何才能更好模拟用户实际操作。

本部分将主要讲解用 selenium 模拟用户登录 WebVPN 的过程。

涉及的内容包括：DriverManager 打开浏览器、get 方法访问页面、找到元素并 send_keys 向搜索框/密码框中发送值、webdriverwait 等待页面加载、自动按键。



### 1.环境配置

首先需要下载 selenium：
```shell
pip3 install selenium
```

然后教程中需要用到的库有：

In [ ]:
import selenium
# 用来创建chrome浏览器
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# 用来等待页面加载
from selenium.webdriver.support.ui import WebDriverWait as wdw
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By # 搜寻元素
from selenium.webdriver.common.keys import Keys # 模拟按键
from selenium.webdriver.remote.webdriver import WebDriver as wd

from selenium.webdriver.common.action_chains import ActionChains as AC # action chains

### 2.建立 chrome 浏览器

以下的代码创建了一个的 chrome 浏览器。

其中，service 参数中的 ChromeDriverManager 在你每次打开浏览器的时候检查你的浏览器版本/驱动是不是最新的，如果不是，就自动下载下来。
否则可能发生 webdriver 不支持你电脑中 Chrome 版本的 error。

In [26]:
d = selenium.webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/luohaowen/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache


### 3.访问目标网页

直接用 get() 方法即可以访问目标网页，在 selenium 中无需考虑 headers 等问题。

比如我们访问清华大学 WebVPN。运行下面的页面，Chrome 自动跳转到 webvpn 的界面：

In [27]:
d.get("https://webvpn.tsinghua.edu.cn/login")

### 4.定位网页元素

我们可以通过 d.find_element() 方法来查找网页中的元素，而 selenium.webdriver.common.by 提供了我们优雅的寻找元素的方式。

比如说我们想查找 WebVPN 中的 username 栏的位置，可以通过 inspect 先找到 username 栏对应的 html 代码位置。发现其 tag 是一个 input，id 是 userCode，所以我们就可以通过以下的方式来查找：

In [28]:
username = d.find_element(By.ID, "user_name")

By 提供的查找元素的方式有通过 ID, TAG_NAME, NAME, CLASS_NAME 等。但是通过这些方法查找你需要保证你所查找的元素是唯一的，或者你知道它是满足同样条件的元素中的第几个元素，这会带来不便。最万金油的方法是通过 `By.XPATH`。可能语法看起来麻烦，但真的用起来会很方便。

这里简单以 username 为例讲解以下 XPATH。username 的 XPATH 为 '//input[@id="userCode"]'。

'//'后的元素为 tagname, '[]'中为元素的 attribute 以及其值（username 的 id attribute 的值即为 userCode）。

当然了，你还可以这样定位 username: '//input[@id="userCode" and @type="text"]'。

上面的这么多已经够用了。想了解更多有关 xpath 的知识可以参见：https://www.w3schools.com/xml/xpath_intro.asp。

找到了 username 之后，我们就可以方便地使用 send_keys method 来模拟向其中输入值：

In [29]:
username.send_keys("2021000000") # plz fill in your username

请你通过同样的方法找到 password 和登录按键 btn。并通过 btn.click() 登入webvpn。

### 5.等待网页加载

网页加载需要一段较为明显的时间，此时如果直接定位元素、发送点击等操作会导致异常。 此时可以使用 webdriverwait 操作进行等待。Selenium 预先封装了许多判据供我们直接调用，它们就是 selenium.webdriver.support.expected_conditions。

也可以通过自己定义和函数的方式来设置等待的判据

常见的几种等待的类型：

visibility_of_element_located 当某个元素出现；

element_to_be_clickable 当某个元素可以点击。

比如你可以在登入 webvpn 后这样进行等待。如果 quick-access-input 的 url 出现，那么我们就认为网页充分加载了。

In [ ]:
url = By.ID, "quick-access-input"
wdw(d, 5).until(EC.visibility_of_element_located(url))

这句话的含义是 d 将 wait 直到 url 可见，最大的等待时间是 5s，如果超出 5s 则认为出现问题。



### 6.Keys and ActionChain

有的时候你需要模拟按键，那么可以直接利用 selenium.webdriver.common.keys.

如果只需要按一个键(比方说在搜索框按回车)，那么直接查找到目标位置 search，然后 search.send_keys(Key.ENTER)。

但如果需要同时按几个键，则需要通过 ActionChain。

比方说如果 webvpn 的搜索框中原来就有内容，你需要先点击搜索框，然后 CONTROL(win)/COMMAND(mac)+A 全选，最后按删除键，你可以这样：

In [ ]:
actions = AC(d) # 创建action_chain对象
# 可以注意一下这里的url为什么加*
# find_element和visibility_of_element_located接受的参数不同哦
actions.move_to_element(d.find_element(*url))
actions.click()
actions. \
    key_down(Keys.CONTROL). \
    send_keys("A"). \
    key_up(Keys.CONTROL). \
    send_keys(Keys.DELETE). \
    perform()

注意，action_chain 将保存所有的 action。直到 actions.perform() 时才依次进行下所有的操作。

在按键之外，一个常见的操作是将页面向下拖动以让其加载出更多内容，并判断是否滑到底部。可以用 execute_script（）执行 js 代码来完成这个操作。

In [ ]:
d.execute_script("scrollBy(0, 2500)")
now_height = d.execute_script("return document.documentElement.scrollHeight") # 返回当前的页面高度

在循环中对比 now_height（下滑后的高度）和 prev_height（上一次的高度）即可完成判断页面是否到底的操作。

但是注意，如果你把浏览器滑走，可能导致加载速度减慢，新元素加载不出来，浏览器还没到底它就以为到底了。

# 三、实际操作过程的注意事项

*1天速成爬虫，从入门到入狱*

*当你放着爬虫爬一晚上，早上起来刚准备大丰收，结果发现你还没睡着的时候它就不动了 :(*

*你也不知道为什么百度就是不返回正确的页面，一直让你做百度安全检测，一直返回网络异常（怎么就你异常人家都好好的捏）*

*油管的页面似乎改过一次版？*

总之，爬虫注意安全性和鲁棒性！！（以及要有耐心）

## 安全

必须要注意的是，爬敏感数据是违法的，所以千万别还没入门就入狱了（x

以及如果你带上了自己的cookie，过于频繁操作是会被封号的。

## 调试

### 1.利用jupyter notebook进行小数据量测试

因为爬取的过程中会出现许多许多的错误，所以非常推荐先在jupyter notebook上先测试自己的代码。

（不过也别弄得太乱，否则整理的时候会很麻烦的....）

### 2.利用logging定位错误

爬取的过程中，要对爬虫的运行状态及产生的异常进行记录，便于后续查看日志，分析运行过程中存在的问题。

这里推荐Python 自带的 logging 模块。对于已经存在的日志，使用例如'cat zhihu.log | grep error | less'就可以查看爬取过程中出现的错误。

可以直接在自己的代码中添加上如下的模块：

In [ ]:
import logging

# 数据格式
fmt = '%(asctime)s.%(msecs)03d [%(levelname)s] %(message)s'
datefmt = '%Y-%m-%d %H:%M:%S'
level = logging.INFO

formatter = logging.Formatter(fmt, datefmt)
logger = logging.getLogger()
logger.setLevel(level)

# 在zhihu.log下输出信息
file = logging.FileHandler('zhihu/zhihu.log', encoding='utf-8')
file.setLevel(level)
file.setFormatter(formatter)
logger.addHandler(file)

# 在控制台console输出信息
console = logging.StreamHandler()
console.setLevel(level)
console.setFormatter(formatter)
logger.addHandler(console)